In [2]:
import pandas as pd
import numpy as np

# Importing the datasets to be consolidated.

In [3]:
# Esk_daily_leakbehav
daily_leaks = pd.read_csv('Esk_daily_leakbehav.csv')
# Renaming the datetime column
daily_leaks.rename(columns={'Unnamed: 0':'Date'}, inplace=True)

In [4]:
daily_leaks.head()

,Date,waterleak_count,meterleak_count,pipeleak_count,valveleak_count
0,2018-07-01,0,0,0,0
1,2018-07-02,0,0,0,0
2,2018-07-03,0,0,0,0
3,2018-07-04,0,0,0,0
4,2018-07-05,0,1,0,0


In [5]:
#Weather data
weather = pd.read_csv('weather.csv')
#weather.head()

# Reshaping pressure data for allignment with other variables

In [6]:
#Pressure data
pressure_us = pd.read_csv('pressure_us.csv')
pressure_ds = pd.read_csv('pressure_ds.csv')

In [7]:
pressure_us.head()

,Datetime,Pressure_us
0,2018-05-04 00:00:00,558.828
1,2018-05-04 00:15:00,558.828
2,2018-05-04 00:30:00,558.828
3,2018-05-04 00:45:00,568.632
4,2018-05-04 01:00:00,558.828


In [8]:
#df2 = pressure_ds.groupby(level=0).sum()

In [9]:
# Converting string object to date time object
pressure_ds['Datetime'] = pd.to_datetime(pressure_ds['Datetime'].str.strip(), format='%Y-%m-%d %H:%M:%S.%f')
pressure_us['Datetime'] = pd.to_datetime(pressure_us['Datetime'].str.strip(), format='%Y-%m-%d %H:%M:%S.%f')
#splitting datetime into day and time 
pressure_ds['Datetime'] = [d.date() for d in pressure_ds['Datetime']]
pressure_us['Datetime'] = [d.date() for d in pressure_us['Datetime']]
#data_int['time'] = [d.time() for d in data_int['DATE_CREATED']]

In [10]:
# Averaging at daily scale
Avg_pressure_ds = pressure_ds.groupby('Datetime').mean()
Avg_pressure_us = pressure_us.groupby('Datetime').mean()

In [11]:
Avg_pressure_us.head()

,Pressure_us
Datetime,
2018-05-04,529.211750
2018-05-05,477.638625
2018-05-06,414.116875
2018-05-07,434.337625
2018-05-08,420.857125


# Selecting time period of interest from a dataframe

In [12]:
# function that takes a series and reindexs it and returns a series according to time period of interest
def rind(series):
    #series = pd.Series(frame['waterleak_count'], index=frame.index)
    # Realigning the leakage data over the time period of interest
    idx = pd.date_range(start='07/01/2018', end='06/30/2019', freq='D')
    series.index = pd.DatetimeIndex(series.index)
    series = series.reindex(idx, fill_value=0)
    return series

In [13]:
# ##Reindexed waterleak counts
#converting dataframe to  a series
series_ds = pd.Series(Avg_pressure_ds['Pressure_ds'], index=Avg_pressure_ds.index)
series_us = pd.Series(Avg_pressure_us['Pressure_us'], index=Avg_pressure_us.index)
#Making use of the reindexing function that was eairlier defined
series_ds = rind(series_ds)
series_us = rind(series_us)
#Converting series back to dataframe
frame_ds = series_ds.to_frame(name='Pressure_ds')
frame_us = series_us.to_frame(name='Pressure_us')
# Reseting the index
frame_ds = frame_ds.reset_index() 
frame_us = frame_us.reset_index()
#Rounding off to one decimal place for consistency with weather data
frame_ds = frame_ds.round(1) 
frame_us = frame_us.round(1)
# Renaming the datetime column
frame_ds.rename(columns={'index':'Date'}, inplace=True)

# Pulling all Variables of interest into one table

In [14]:
# Joining the data frames together
Comb = frame_ds 
Comb['Pressure_us'] = frame_us.Pressure_us
Comb['Minimum_Temp'] = weather.Minimum_Temp
Comb['Maximum_Temp'] = weather.Maximum_Temp
Comb['Average_Temp'] = weather.Average_Temp
Comb['Rainfall'] = weather.Rainfall
Comb['meterleak_count'] = daily_leaks.meterleak_count
Comb['pipeleak_count'] = daily_leaks.pipeleak_count
Comb['valveleak_count'] = daily_leaks.valveleak_count
Comb['waterleak_count'] = daily_leaks.waterleak_count

In [15]:
Comb.shape

(365, 11)

In [16]:
# Writing new table to a different csv file for safety.
Comb.to_csv(r'Esk_leakdata_1.csv', index = False)

In [18]:
df = pd.read_csv('Esk_leakdata_1.csv')

In [19]:
df.head()

,Date,Pressure_ds,Pressure_us,Minimum_Temp,Maximum_Temp,Average_Temp,Rainfall,meterleak_count,pipeleak_count,valveleak_count,waterleak_count
0,2018-07-01,317.4,420.1,12.3,25.3,15.1,0.0,0,0,0,0
1,2018-07-02,323.3,425.8,17.3,28.6,19.7,0.0,0,0,0,0
2,2018-07-03,326.3,433.6,9.6,20.9,18.7,0.0,0,0,0,0
3,2018-07-04,314.0,419.5,7.6,22.3,10.9,0.0,0,0,0,0
4,2018-07-05,333.8,467.5,9.6,22.3,13.4,0.6,1,0,0,0
